In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws

In [2]:
import cloudknot as ck

In [3]:
def download_and_track(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    aws.download_s3(filename, op.split(filename)[1])
    
    outfile = 'Traj_' + op.split(filename)[1].split('.')[0] + '.csv'

    ij.track(op.split(filename)[1], outfile, template=None, fiji_bin=None, radius=3.5, threshold=0., 
          do_median_filtering=True, quality=30.0, median_intensity=300.0, snr=0.0, 
          linking_max_distance=7.0, gap_closing_max_distance=10.0, max_frame_gap=2,
          track_displacement=10.0)
    
    aws.upload_s3(outfile, op.join(op.split(filename)[0], outfile))

In [4]:
download_and_track('01_18_Experiment/P1_S1_L_0000_0_0.tif')

/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmprUceg7.py


OSError: [Errno 2] No such file or directory